In [1]:
from pinecone import Pinecone, PodSpec 
from langchain.document_loaders import PyMuPDFLoader


from IPython.display import Markdown, display

f:\Jupyter\Upwork\DocumentQA\DocumentQA\venv\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [ ]:

from hybrid_search import HybridPinecone

### Initialize Vector Database

In [5]:
pc = HybridPinecone(
    api_key = "3c9fd500-de4d-40ed-8c77-12302b71e8ce",  # app.pinecone.io
    environment = "gcp-starter"
)

NameError: name 'HybridPinecone' is not defined

In [ ]:
index_name = "hybrid-test"

# create the index
pc.create_index(
   name = index_name,
   dimension = 384,  # dimensionality of dense model
   spec=PodSpec(environment="gcp-starter"),
   metric = "dotproduct",
)

### Load Data
We use langchain PDF loaders to read pdfs. 

In [6]:
filepath = "data/E1. ExngTextOnly.pdf"
loader = PyMuPDFLoader(filepath)
documents = loader.load()

In [7]:
contexts = [document.page_content for document in documents]

In [ ]:
from transformers import BertTokenizerFast
# load bert tokenizer from huggingface
tokenizer = BertTokenizerFast.from_pretrained(
    'bert-base-uncased'
)


In [8]:

inputs = tokenizer(
    contexts[0], padding=True, truncation=True,
    max_length=512
)
inputs.keys()

# %%
# extract the input ids
input_ids = inputs['input_ids']
input_ids

[101,
 2040,
 2024,
 1996,
 4587,
 2111,
 2017,
 2113,
 1029,
 2054,
 2024,
 1996,
 4587,
 2477,
 2017,
 2031,
 1006,
 1045,
 1007,
 1999,
 2115,
 2160,
 1010,
 1006,
 2462,
 1007,
 1999,
 2115,
 2103,
 1010,
 2237,
 2030,
 2352,
 1029,
 2129,
 2214,
 2024,
 2027,
 1029,
 2031,
 2017,
 2412,
 6257,
 2008,
 2017,
 2020,
 3080,
 1029,
 2031,
 2017,
 6257,
 2008,
 2017,
 2071,
 4982,
 2039,
 1999,
 1037,
 9241,
 1029,
 2720,
 16172,
 2180,
 2912,
 4269,
 2011,
 1999,
 26703,
 2180,
 2912,
 1011,
 6819,
 2618,
 1010,
 2029,
 3084,
 2111,
 3920,
 1012,
 2021,
 2180,
 2912,
 1011,
 6819,
 2618,
 2003,
 2205,
 2844,
 1012,
 2061,
 2070,
 2111,
 10436,
 1010,
 2138,
 2037,
 2287,
 4150,
 15718,
 999,
 2028,
 2711,
 2941,
 4150,
 15718,
 12021,
 1011,
 2698,
 1010,
 2029,
 2965,
 2002,
 1521,
 1055,
 2288,
 2000,
 3524,
 12021,
 1011,
 2698,
 2086,
 2077,
 2002,
 2064,
 2272,
 2067,
 1012,
 2720,
 16172,
 2180,
 2912,
 2442,
 1999,
 15338,
 1037,
 2047,
 2518,
 1012,
 1012,
 1012,
 2720,
 2180,